In [2]:
##importando pacotes
import pandas as pd
import numpy as np
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl
import csv
import re

In [3]:
os.chdir('G:\\\\Drives compartilhados\\\\República.org\\\\4. Equipes\\\\Dados e Comunicação\\\\DADOS\\\\415 - Repositório de Dados\\\\Dados para subir\\\\Dados Brutos\\\\INEP\\\\Esforço Docente')


In [4]:
df = pd.read_csv('INEP_esforco_docente.csv', sep=';', encoding='latin-1')
df

,ano,nome_regiao,sigla_uf,id_municipio,nome_municipio,id_escola,nome_escola,localizacao,dependencia_adm,ef_total_grupo1,...,ef_finais_grupo3,ef_finais_grupo4,ef_finais_grupo5,ef_finais_grupo6,em_grupo1,em_grupo2,em_grupo3,em_grupo4,em_grupo5,em_grupo6
0,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11022558,EIEEF HAP BITT TUPARI,Rural,Estadual,"100,0",...,"0,0","0,0","0,0","0,0",NaN,NaN,NaN,NaN,NaN,NaN
1,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024291,EMMEF 07 DE SETEMBRO,Rural,Municipal,"100,0",...,"0,0","0,0","0,0","0,0",NaN,NaN,NaN,NaN,NaN,NaN
2,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024313,EMMEF AFONSO AURELIO PORTO,Rural,Municipal,"100,0",...,"0,0","0,0","0,0","0,0",NaN,NaN,NaN,NaN,NaN,NaN
3,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024372,EMEIEF ANA NERY,Rural,Municipal,"20,0",...,"0,0","71,4","0,0","0,0",NaN,NaN,NaN,NaN,NaN,NaN
4,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024402,EMMEF ANTONIO FRANCISCO LISBOA,Rural,Municipal,"100,0",...,"0,0","0,0","0,0","0,0",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1384272,2022,Centro-Oeste,DF,5300108,Brasília,53084020,ESC PEQUENO ENCANTO,Urbana,Privada,"20,0",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1384273,2022,Centro-Oeste,DF,5300108,Brasília,53084039,COL ANCHIETA,Urbana,Privada,"9,1",...,"28,6","35,7","14,3","0,0",NaN,NaN,NaN,NaN,NaN,NaN
1384274,2022,Centro-Oeste,DF,5300108,Brasília,53084055,COL MAPA,Urbana,Privada,"29,4",...,"54,5","36,4","0,0","0,0","0,0","7,6","38,5","30,8","15,4","7,7"
1384275,2022,Centro-Oeste,DF,5300108,Brasília,53085000,COBIAN - COL BIANGULO,Urbana,Privada,"7,6",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df = df.rename(columns= {'ef_total_grupo1':'ef.total_grupo1',
                         'ef_total_grupo2':'ef.total_grupo2',
                         'ef_total_grupo3':'ef.total_grupo3',
                         'ef_total_grupo4':'ef.total_grupo4',
                         'ef_total_grupo5':'ef.total_grupo5',
                         'ef_total_grupo6':'ef.total_grupo6',
                         'ef_iniciais_grupo1':'ef.iniciais_grupo1', 
                         'ef_iniciais_grupo2':'ef.iniciais_grupo2',
                         'ef_iniciais_grupo3':'ef.iniciais_grupo3',
                         'ef_iniciais_grupo4':'ef.iniciais_grupo4',
                         'ef_iniciais_grupo5':'ef.iniciais_grupo5',
                         'ef_iniciais_grupo6':'ef.iniciais_grupo6',
                         'ef_finais_grupo1':'ef.finais_grupo1',
                         'ef_finais_grupo2':'ef.finais_grupo2',
                         'ef_finais_grupo3':'ef.finais_grupo3',
                         'ef_finais_grupo4':'ef.finais_grupo4',
                         'ef_finais_grupo5':'ef.finais_grupo5', 
                         'ef_finais_grupo6':'ef.finais_grupo6'})

In [32]:
df.columns

Index(['ano', 'nome_regiao', 'sigla_uf', 'id_municipio', 'nome_municipio',
       'id_escola', 'nome_escola', 'localizacao', 'dependencia_adm',
       'ef.total_grupo1', 'ef.total_grupo2', 'ef.total_grupo3',
       'ef.total_grupo4', 'ef.total_grupo5', 'ef.total_grupo6',
       'ef.iniciais_grupo1', 'ef.iniciais_grupo2', 'ef.iniciais_grupo3',
       'ef.iniciais_grupo4', 'ef.iniciais_grupo5', 'ef.iniciais_grupo6',
       'ef.finais_grupo1', 'ef.finais_grupo2', 'ef.finais_grupo3',
       'ef.finais_grupo4', 'ef.finais_grupo5', 'ef.finais_grupo6', 'em_grupo1',
       'em_grupo2', 'em_grupo3', 'em_grupo4', 'em_grupo5', 'em_grupo6'],
      dtype='object')

In [7]:
df.head(5)

,ano,nome_regiao,sigla_uf,id_municipio,nome_municipio,id_escola,nome_escola,localizacao,dependencia_adm,grupos_docentes_por_ano_escolar,per_valor
0,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11022558,EIEEF HAP BITT TUPARI,Rural,Estadual,ef.total_grupo1,"100,0"
1,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024291,EMMEF 07 DE SETEMBRO,Rural,Municipal,ef.total_grupo1,"100,0"
2,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024313,EMMEF AFONSO AURELIO PORTO,Rural,Municipal,ef.total_grupo1,"100,0"
3,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024372,EMEIEF ANA NERY,Rural,Municipal,ef.total_grupo1,"20,0"
4,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024402,EMMEF ANTONIO FRANCISCO LISBOA,Rural,Municipal,ef.total_grupo1,"100,0"


In [6]:
df = df.melt(id_vars =['ano', 'nome_regiao', 'sigla_uf', 'id_municipio','nome_municipio','id_escola','nome_escola','localizacao','dependencia_adm'], 
        value_vars =['ef.total_grupo1', 'ef.total_grupo2', 'ef.total_grupo3',
       'ef.total_grupo4', 'ef.total_grupo5', 'ef.total_grupo6',
       'ef.iniciais_grupo1', 'ef.iniciais_grupo2', 'ef.iniciais_grupo3',
       'ef.iniciais_grupo4', 'ef.iniciais_grupo5', 'ef.iniciais_grupo6',
       'ef.finais_grupo1', 'ef.finais_grupo2', 'ef.finais_grupo3',
       'ef.finais_grupo4', 'ef.finais_grupo5', 'ef.finais_grupo6', 'em_grupo1',
       'em_grupo2', 'em_grupo3', 'em_grupo4', 'em_grupo5', 'em_grupo6'], 
        var_name ='grupos_docentes_por_ano_escolar', 
        value_name ='per_valor')
df


,ano,nome_regiao,sigla_uf,id_municipio,nome_municipio,id_escola,nome_escola,localizacao,dependencia_adm,grupos_docentes_por_ano_escolar,per_valor
0,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11022558,EIEEF HAP BITT TUPARI,Rural,Estadual,ef.total_grupo1,"100,0"
1,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024291,EMMEF 07 DE SETEMBRO,Rural,Municipal,ef.total_grupo1,"100,0"
2,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024313,EMMEF AFONSO AURELIO PORTO,Rural,Municipal,ef.total_grupo1,"100,0"
3,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024372,EMEIEF ANA NERY,Rural,Municipal,ef.total_grupo1,"20,0"
4,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024402,EMMEF ANTONIO FRANCISCO LISBOA,Rural,Municipal,ef.total_grupo1,"100,0"
...,...,...,...,...,...,...,...,...,...,...,...
33222643,2022,Centro-Oeste,DF,5300108,Brasília,53084020,ESC PEQUENO ENCANTO,Urbana,Privada,em_grupo6,NaN
33222644,2022,Centro-Oeste,DF,5300108,Brasília,53084039,COL ANCHIETA,Urbana,Privada,em_grupo6,NaN
33222645,2022,Centro-Oeste,DF,5300108,Brasília,53084055,COL MAPA,Urbana,Privada,em_grupo6,"7,7"
33222646,2022,Centro-Oeste,DF,5300108,Brasília,53085000,COBIAN - COL BIANGULO,Urbana,Privada,em_grupo6,NaN


In [34]:
df['grupos_docentes_por_ano_escolar'].unique()

array(['ef.total_grupo1', 'ef.total_grupo2', 'ef.total_grupo3',
       'ef.total_grupo4', 'ef.total_grupo5', 'ef.total_grupo6',
       'ef.iniciais_grupo1', 'ef.iniciais_grupo2', 'ef.iniciais_grupo3',
       'ef.iniciais_grupo4', 'ef.iniciais_grupo5', 'ef.iniciais_grupo6',
       'ef.finais_grupo1', 'ef.finais_grupo2', 'ef.finais_grupo3',
       'ef.finais_grupo4', 'ef.finais_grupo5', 'ef.finais_grupo6',
       'em_grupo1', 'em_grupo2', 'em_grupo3', 'em_grupo4', 'em_grupo5',
       'em_grupo6'], dtype=object)

In [8]:
df1= df['grupos_docentes_por_ano_escolar'].str.split('_', n=1, expand=True)
df1

,0,1
0,ef.total,grupo1
1,ef.total,grupo1
2,ef.total,grupo1
3,ef.total,grupo1
4,ef.total,grupo1
...,...,...
33222643,em,grupo6
33222644,em,grupo6
33222645,em,grupo6
33222646,em,grupo6


In [11]:
df1.columns = ['ano_escolar', 'docente_grupos']
df1

,ano_escolar,docente_grupos
0,ef.total,grupo1
1,ef.total,grupo1
2,ef.total,grupo1
3,ef.total,grupo1
4,ef.total,grupo1
...,...,...
33222643,em,grupo6
33222644,em,grupo6
33222645,em,grupo6
33222646,em,grupo6


In [37]:
df1['ano_escolar'].unique()

array(['ef.total', 'ef.iniciais', 'ef.finais', 'em'], dtype=object)

In [12]:
def transformar (x):
    x = re.sub(r"\bem\b","Ensino Médio", x)
    x = re.sub(r"\bef.total\b","Ensino Fundamental - Todos os anos", x)
    x = re.sub(r"\bef.iniciais\b","Ensino Fundamental - Anos Iniciais", x)
    x = re.sub(r"\bef.finais\b","Ensino Fundamental - Anos Finais", x)
    x = re.sub(r"\bgrupo1\b","Grupo 1", x)
    x = re.sub(r"\bgrupo2\b","Grupo 2", x)
    x = re.sub(r"\bgrupo3\b","Grupo 3", x)
    x = re.sub(r"\bgrupo4\b","Grupo 4", x)
    x = re.sub(r"\bgrupo5\b","Grupo 5", x)
    x = re.sub(r"\bgrupo6\b","Grupo 6", x)
    return x

In [13]:
df1['ano_escolar'] = df1['ano_escolar'].apply(transformar)

In [14]:
df1['docente_grupos'] = df1['docente_grupos'].apply(transformar)

In [41]:
df1

,ano_escolar,docente_grupos
0,Ensino Fundamental - Todos os anos,Grupo 1
1,Ensino Fundamental - Todos os anos,Grupo 1
2,Ensino Fundamental - Todos os anos,Grupo 1
3,Ensino Fundamental - Todos os anos,Grupo 1
4,Ensino Fundamental - Todos os anos,Grupo 1
...,...,...
33222643,Ensino Médio,Grupo 6
33222644,Ensino Médio,Grupo 6
33222645,Ensino Médio,Grupo 6
33222646,Ensino Médio,Grupo 6


In [43]:
df1['ano_escolar'].unique()

array(['Ensino Fundamental - Todos os anos',
       'Ensino Fundamental - Anos Iniciais',
       'Ensino Fundamental - Anos Finais', 'Ensino Médio'], dtype=object)

In [15]:
df = pd.concat([df, df1], axis=1)

In [16]:
df = df.drop(columns='grupos_docentes_por_ano_escolar')

In [17]:
df = df[['ano', 'nome_regiao', 'sigla_uf', 'id_municipio', 'nome_municipio',
       'id_escola', 'nome_escola', 'localizacao', 'dependencia_adm','ano_escolar', 'docente_grupos', 'per_valor']]
df

,ano,nome_regiao,sigla_uf,id_municipio,nome_municipio,id_escola,nome_escola,localizacao,dependencia_adm,ano_escolar,docente_grupos,per_valor
0,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11022558,EIEEF HAP BITT TUPARI,Rural,Estadual,Ensino Fundamental - Todos os anos,Grupo 1,"100,0"
1,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024291,EMMEF 07 DE SETEMBRO,Rural,Municipal,Ensino Fundamental - Todos os anos,Grupo 1,"100,0"
2,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024313,EMMEF AFONSO AURELIO PORTO,Rural,Municipal,Ensino Fundamental - Todos os anos,Grupo 1,"100,0"
3,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024372,EMEIEF ANA NERY,Rural,Municipal,Ensino Fundamental - Todos os anos,Grupo 1,"20,0"
4,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024402,EMMEF ANTONIO FRANCISCO LISBOA,Rural,Municipal,Ensino Fundamental - Todos os anos,Grupo 1,"100,0"
...,...,...,...,...,...,...,...,...,...,...,...,...
33222643,2022,Centro-Oeste,DF,5300108,Brasília,53084020,ESC PEQUENO ENCANTO,Urbana,Privada,Ensino Médio,Grupo 6,NaN
33222644,2022,Centro-Oeste,DF,5300108,Brasília,53084039,COL ANCHIETA,Urbana,Privada,Ensino Médio,Grupo 6,NaN
33222645,2022,Centro-Oeste,DF,5300108,Brasília,53084055,COL MAPA,Urbana,Privada,Ensino Médio,Grupo 6,"7,7"
33222646,2022,Centro-Oeste,DF,5300108,Brasília,53085000,COBIAN - COL BIANGULO,Urbana,Privada,Ensino Médio,Grupo 6,NaN


In [18]:
df[df['id_escola']=="11022558"]

,ano,nome_regiao,sigla_uf,id_municipio,nome_municipio,id_escola,nome_escola,localizacao,dependencia_adm,ano_escolar,docente_grupos,per_valor


In [19]:
df['per_valor'] = df['per_valor'].str.replace(',','.').astype(float)

In [20]:
df

,ano,nome_regiao,sigla_uf,id_municipio,nome_municipio,id_escola,nome_escola,localizacao,dependencia_adm,ano_escolar,docente_grupos,per_valor
0,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11022558,EIEEF HAP BITT TUPARI,Rural,Estadual,Ensino Fundamental - Todos os anos,Grupo 1,100.0
1,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024291,EMMEF 07 DE SETEMBRO,Rural,Municipal,Ensino Fundamental - Todos os anos,Grupo 1,100.0
2,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024313,EMMEF AFONSO AURELIO PORTO,Rural,Municipal,Ensino Fundamental - Todos os anos,Grupo 1,100.0
3,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024372,EMEIEF ANA NERY,Rural,Municipal,Ensino Fundamental - Todos os anos,Grupo 1,20.0
4,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024402,EMMEF ANTONIO FRANCISCO LISBOA,Rural,Municipal,Ensino Fundamental - Todos os anos,Grupo 1,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...
33222643,2022,Centro-Oeste,DF,5300108,Brasília,53084020,ESC PEQUENO ENCANTO,Urbana,Privada,Ensino Médio,Grupo 6,NaN
33222644,2022,Centro-Oeste,DF,5300108,Brasília,53084039,COL ANCHIETA,Urbana,Privada,Ensino Médio,Grupo 6,NaN
33222645,2022,Centro-Oeste,DF,5300108,Brasília,53084055,COL MAPA,Urbana,Privada,Ensino Médio,Grupo 6,7.7
33222646,2022,Centro-Oeste,DF,5300108,Brasília,53085000,COBIAN - COL BIANGULO,Urbana,Privada,Ensino Médio,Grupo 6,NaN


In [21]:
df.dtypes

ano                  int64
nome_regiao         object
sigla_uf            object
id_municipio         int64
nome_municipio      object
id_escola            int64
nome_escola         object
localizacao         object
dependencia_adm     object
ano_escolar         object
docente_grupos      object
per_valor          float64
dtype: object

In [22]:
client = bigquery.Client()

In [23]:
dataset_ref = client.dataset('Datalake')

In [24]:
schema = [bigquery.SchemaField('ano', 'INTEGER', description= 'Ano de referência da observação'),
          bigquery.SchemaField('nome_regiao', 'STRING', description= 'Região do Brasil abrangente a observação'),
          bigquery.SchemaField('sigla_uf', 'STRING', description= 'Sigla da Unidade da Federação referente a observação'),
          bigquery.SchemaField('id_municipio', 'INTEGER', description= 'Identificador do município pelo IBGE'),
          bigquery.SchemaField('nome_municipio', 'STRING', description= 'Nome do município da observação'),                    
          bigquery.SchemaField('id_escola', 'INTEGER', description= 'Identificação da escola'),                    
          bigquery.SchemaField('nome_escola', 'STRING', description= 'Nome da escola observada'),                    
          bigquery.SchemaField('localizacao', 'STRING', description= 'Se a área da observação é urbana ou rural'),
          bigquery.SchemaField('dependencia_adm', 'STRING', description= 'Dependência admnistrativa da observação (privada ou pública)'),
          bigquery.SchemaField('ano_escolar', 'STRING', description= 'Ano escolar referente ao qual docente leciona'),
          bigquery.SchemaField('docente_grupos', 'STRING', description= 'Cinco categorias de adequação da formação dos docentes em relação à disciplina que leciona'),
          bigquery.SchemaField('per_valor', 'FLOAT', description= 'Percentual de docentes que lecionam na respectiva agregação em cada um dos níveis da escala do indicador'),
]

In [25]:
table_ref = dataset_ref.table('INEP_esforco_docente')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()

LoadJob<project=repositoriodedadosgpsp, location=US, id=01cd9b32-8c8a-462f-9496-1add1f22201b>